In [12]:
pip install imbalanced-learn

     |████████████████████████████████| 167 kB 3.0 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [44]:
import pandas as pd
url="https://bd29ee0e-54ab-4daa-9671-d153865d1620.usrfiles.com/ugd/bd29ee_885fd28f4a654b0cbf2dee1e46c7c867.csv"
df=pd.read_csv(url,header=None)
df.head()

,0,1,2,3,4,5,6,7
0,35.5,no,yes,no,no,no,no,no
1,35.9,no,no,yes,yes,yes,yes,no
2,35.9,no,yes,no,no,no,no,no
3,36.0,no,no,yes,yes,yes,yes,no
4,36.0,no,yes,no,no,no,no,no


In [45]:
df.shape

(120, 8)

In [46]:
df.groupby(df.iloc[:,6]).count().T

6,no,yes
0,61,59
1,61,59
2,61,59
3,61,59
4,61,59
5,61,59
6,61,59
7,61,59


In [47]:
df.groupby(df.iloc[:,7]).count().T

7,no,yes
0,70,50
1,70,50
2,70,50
3,70,50
4,70,50
5,70,50
6,70,50
7,70,50


In [48]:
objects=df.iloc[:,1:8]
objects.head()

,1,2,3,4,5,6,7
0,no,yes,no,no,no,no,no
1,no,no,yes,yes,yes,yes,no
2,no,yes,no,no,no,no,no
3,no,no,yes,yes,yes,yes,no
4,no,yes,no,no,no,no,no


In [49]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()
objects=objects.apply(le.fit_transform)
objects.head()

,1,2,3,4,5,6,7
0,0,1,0,0,0,0,0
1,0,0,1,1,1,1,0
2,0,1,0,0,0,0,0
3,0,0,1,1,1,1,0
4,0,1,0,0,0,0,0


In [50]:
temp=pd.DataFrame(df.iloc[:,0])
df=pd.concat([temp,objects],axis=1)
df.head()

,0,1,2,3,4,5,6,7
0,35.5,0,1,0,0,0,0,0
1,35.9,0,0,1,1,1,1,0
2,35.9,0,1,0,0,0,0,0
3,36.0,0,0,1,1,1,1,0
4,36.0,0,1,0,0,0,0,0


In [51]:
X=df.iloc[:,0:6]
X.head()

,0,1,2,3,4,5
0,35.5,0,1,0,0,0
1,35.9,0,0,1,1,1
2,35.9,0,1,0,0,0
3,36.0,0,0,1,1,1
4,36.0,0,1,0,0,0


In [52]:
y1=pd.DataFrame(df.iloc[:,6])
y1.head()

,6
0,0
1,1
2,0
3,1
4,0


In [53]:
y2=pd.DataFrame(df.iloc[:,7])
y2.head()

,7
0,0
1,0
2,0
3,0
4,0


In [54]:
from sklearn.preprocessing import StandardScaler
scaler=StandardScaler()
X=pd.DataFrame(scaler.fit_transform(X))
X.head()

,0,1,2,3,4,5
0,-1.779796,-0.564519,0.845154,-1.414214,-0.983470,-0.845154
1,-1.558989,-0.564519,-1.183216,0.707107,1.016808,1.183216
2,-1.558989,-0.564519,0.845154,-1.414214,-0.983470,-0.845154
3,-1.503788,-0.564519,-1.183216,0.707107,1.016808,1.183216
4,-1.503788,-0.564519,0.845154,-1.414214,-0.983470,-0.845154


In [55]:
from imblearn.ensemble import BalancedRandomForestClassifier
import numpy as np
from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

In [59]:
BForest1=BalancedRandomForestClassifier(n_estimators=325,criterion="gini",max_depth=7,min_samples_split=3,
                                       min_samples_leaf=1,max_features="log2",bootstrap=False,
                                       sampling_strategy="auto",replacement=False,n_jobs=-1,warm_start=False,
                                       random_state=0)
scores=cross_val_score(BForest1,X,np.asarray(y1).reshape(y1.shape[0],),cv=10)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  1.0 +/- 0.0


In [60]:
from sklearn.metrics import roc_auc_score as auc
from sklearn.model_selection import cross_val_predict
y_pred = cross_val_predict(BForest1, X,np.asarray(y1).reshape(y1.shape[0],), cv=10)
AUC = auc(y1, y_pred)
print("AUC for the prediction of the first disease: ",AUC)

AUC for the prediction of the first disease:  1.0


In [61]:
from sklearn.metrics import confusion_matrix
print("Confusion matrix for predicting the first disease: \n",confusion_matrix(y1,y_pred))

Confusion matrix for predicting the first disease: 
 [[61  0]
 [ 0 59]]


In [62]:
BForest2=BalancedRandomForestClassifier(n_estimators=200,criterion="entropy",max_depth=8,min_samples_split=4,
                                       min_samples_leaf=2,max_features="auto",bootstrap=True,
                                       sampling_strategy="all",replacement=True,n_jobs=-1,warm_start=False,
                                       random_state=0)
scores=cross_val_score(BForest2,X,np.asarray(y2).reshape(y2.shape[0],),cv=16)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.984375 +/- 0.06051536478449089


In [63]:
y_pred = cross_val_predict(BForest2, X,np.asarray(y2).reshape(y2.shape[0],), cv=16)
AUC = auc(y2, y_pred)
print("AUC for the prediction of the second disease: ",AUC)

AUC for the prediction of the second disease:  0.98


In [64]:
print("Confusion matrix for predicting the second disease: \n",confusion_matrix(y2,y_pred))

Confusion matrix for predicting the second disease: 
 [[70  0]
 [ 2 48]]


In [65]:
y_pred=pd.DataFrame(y_pred).rename(columns={0:"Predicted label"})
y2=y2.rename(columns={7:"Actual label"})

In [66]:
compare=pd.concat([y2,y_pred],axis=1)
compare=pd.DataFrame(compare["Actual label"]==compare["Predicted label"]).rename(columns={0:"Comparison"})
compare[compare["Comparison"]==False]

,Comparison
60,False
61,False


In [67]:
df2=pd.concat([X,y2],axis=1)
df2=df2.drop(index=[60,61])
df2

,0,1,2,3,4,5,Actual label
0,-1.779796,-0.564519,0.845154,-1.414214,-0.983470,-0.845154,0
1,-1.558989,-0.564519,-1.183216,0.707107,1.016808,1.183216,0
2,-1.558989,-0.564519,0.845154,-1.414214,-0.983470,-0.845154,0
3,-1.503788,-0.564519,-1.183216,0.707107,1.016808,1.183216,0
4,-1.503788,-0.564519,0.845154,-1.414214,-0.983470,-0.845154,0
...,...,...,...,...,...,...,...
115,1.477107,-0.564519,0.845154,0.707107,-0.983470,1.183216,1
116,1.532308,-0.564519,-1.183216,-1.414214,-0.983470,-0.845154,0
117,1.532308,1.771421,0.845154,-1.414214,1.016808,-0.845154,1
118,1.532308,-0.564519,0.845154,0.707107,-0.983470,1.183216,1


In [68]:
df2.groupby("Actual label").count().T

Actual label,0,1
0,70,48
1,70,48
2,70,48
3,70,48
4,70,48
5,70,48


In [69]:
X=df2.drop(["Actual label"],axis=1)
X.head()

,0,1,2,3,4,5
0,-1.779796,-0.564519,0.845154,-1.414214,-0.983470,-0.845154
1,-1.558989,-0.564519,-1.183216,0.707107,1.016808,1.183216
2,-1.558989,-0.564519,0.845154,-1.414214,-0.983470,-0.845154
3,-1.503788,-0.564519,-1.183216,0.707107,1.016808,1.183216
4,-1.503788,-0.564519,0.845154,-1.414214,-0.983470,-0.845154


In [70]:
y2=df2[["Actual label"]]
y2.head()

,Actual label
0,0
1,0
2,0
3,0
4,0


In [71]:
BForest2=BalancedRandomForestClassifier(n_estimators=200,criterion="entropy",max_depth=8,min_samples_split=4,
                                       min_samples_leaf=2,max_features="auto",bootstrap=True,
                                       sampling_strategy="all",replacement=True,n_jobs=-1,warm_start=False,
                                       random_state=0)
scores=cross_val_score(BForest2,X,np.asarray(y2).reshape(y2.shape[0],),cv=16)
print("Accuracy: ",scores.mean(),"+/-",scores.std())

Accuracy:  0.9921875 +/- 0.030257682392245445


In [72]:
y_pred = cross_val_predict(BForest2, X,np.asarray(y2).reshape(y2.shape[0],), cv=16)
AUC = auc(y2, y_pred)
print("AUC for the prediction of the second disease: ",AUC)

AUC for the prediction of the second disease:  0.9895833333333333


In [73]:
print("Confusion matrix for predicting the second disease: \n",confusion_matrix(y2,y_pred))

Confusion matrix for predicting the second disease: 
 [[70  0]
 [ 1 47]]


In [74]:
y_pred=pd.DataFrame(cross_val_predict(BForest2, X, np.asarray(y2).reshape(y2.shape[0],), 
                                      cv=16,method='predict_proba')).rename(columns={0:"pred_0",1:"pred_1"})
y_pred


,pred_0,pred_1
0,0.996667,0.003333
1,1.000000,0.000000
2,0.996667,0.003333
3,1.000000,0.000000
4,0.996667,0.003333
...,...,...
113,0.013649,0.986351
114,0.842446,0.157554
115,0.006313,0.993687
116,0.003874,0.996126


In [75]:
y_probs1=y_pred[["pred_1"]]

In [76]:
import numpy as np
thresholds=np.arange(0,1,0.001)

In [77]:
def to_labels(pos_probs,threshold):
    return (pos_probs >= threshold).astype('int')

In [78]:
from sklearn.metrics import f1_score
scores=[f1_score(y2, to_labels(y_probs1,t),average="micro") for t in thresholds]

In [79]:
ix=np.argmax(scores)
print((thresholds[ix],scores[ix]))

(0.158, 1.0)


In [80]:
y_pred["pred_0"]=np.where((y_pred.pred_0 >=0.158),1,y_pred.pred_0)
y_pred["pred_0"]=np.where((y_pred.pred_0 < 0.158),0,y_pred.pred_0)
y_pred["pred_1"]=np.where((y_pred.pred_1 >=0.158),1,y_pred.pred_1)
y_pred["pred_1"]=np.where((y_pred.pred_1 <0.158),0,y_pred.pred_1)

In [81]:
len11=len(y_pred[y_pred["pred_1"]==1])
print("The number of predicted positive cases: ",len11)

The number of predicted positive cases:  48


In [84]:
y_pred["pred_0"]=np.where((y_pred.pred_0 == y_pred.pred_1),0,y_pred.pred_0)

In [85]:
len00=len(y_pred[y_pred["pred_0"]==1])
print("The number of predicted negative cases: ",len00)

The number of predicted negative cases:  70


In [86]:
from sklearn.preprocessing import OneHotEncoder
enc=OneHotEncoder(handle_unknown="ignore")
y_true=pd.DataFrame(enc.fit_transform(y2).toarray())

In [87]:
y_pred=enc.inverse_transform(y_pred)

In [88]:
print("AUC after adjusting for the threshold: ",auc(y2,y_pred))

AUC after adjusting for the threshold:  1.0


In [89]:
print("Confusion_matrix after adjusting the threshold: \n",confusion_matrix(y2,y_pred))

Confusion_matrix after adjusting the threshold: 
 [[70  0]
 [ 0 48]]
